# Detection of Bruxism events in Tinnitus patients polysomnographic data
This notebook will 
- load EMG channels of polusomnographic data
- detect the EMG bursts in a unsupervised way
- classify EMG bursts as different bruxism events
- give insights on the bruxism events

In [1]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import mne
import scipy
import seaborn as sns
from tinnsleep.config import Config
from tinnsleep.data import CreateRaw, RawToEpochs_sliding, CleanAnnotations, AnnotateRaw_sliding
from tinnsleep.classification import AmplitudeThresholding
from tinnsleep.signal import rms
from tinnsleep.visualization import plotTimeSeries, plotAnnotations, zoom_effect

from IPython.core.display import display
from ipywidgets import widgets
print("Config loaded")


Config loaded


## Load, filter, and prepare data

In [2]:
filename = Config.bruxisme_files[0]  # load file from config
picks_chan = ['1', '2']           # subset of EMG electrodes

raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading
raw  = CreateRaw(raw[picks_chan][0], picks_chan, ch_types=['emg'])        # pick channels and load
raw  = raw.load_data()  # load data into memory 
print("Data loaded")

raw  = raw.filter(20., 99., n_jobs=4, 
                  fir_design='firwin', filter_length='auto', phase='zero-double',
                  picks=picks_chan)
ch_names = raw.info["ch_names"]
print("Data filtered")

tmin = raw.times[0]                     
tmax = raw.times[-1]

# remove the two first hours and last hour
croptimes=dict(tmin=raw.times[0]+3600*2, tmax=raw.times[-1]-3600)
raw.crop(**croptimes)
offset = raw.times[0]
print(f"keeping {(raw.times[-1]-raw.times[0])/3600:0.2f} hours of recording out of {(tmax-tmin)/3600:0.2f} hours")

Extracting EDF parameters from /Users/louis/Data/SIOPI/bruxisme/1BA07_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-2-bf07b369b378>:4: RuntimeWarning: 7 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Intensit? lumine', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


Data loaded
Data filtered
keeping 8.03 hours of recording out of 11.03 hours


## Epoching data

In [3]:
sfreq = raw.info["sfreq"]
window_length = 0.25                    # in seconds
duration = int(window_length * sfreq)   # in samples
interval = duration                     # no overlapping
epochs = RawToEpochs_sliding(raw, duration=duration, interval=interval)
print(f"Epochs done, shape {epochs.shape}")

Epochs done, shape (115700, 2, 50)


## Classifying epochs and annotate raw

In [6]:
# compute the sum of power over electrodes and samples in each window
pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=2)
X        = rms(epochs)
labels   = pipeline.fit_predict(X)
print(f"bursts count: {np.sum(labels)}/{len(labels)} ({np.sum(labels) / len(labels) * 100:.2f}%)")
print(f"bursts time: {np.sum(labels) * window_length} seconds")

dict_annotations = {1: "burst"}
annotations = []
for k, label in enumerate(labels):
    if label > 0:
        annotations.append(dict(
            onset=k*interval/sfreq,
            duration=duration/sfreq,
            description=dict_annotations[label],
            orig_time=offset
        )
            
        )

bursts count: 7417/115700 (6.41%)
bursts time: 1854.25 seconds


## Display Annotations

In [7]:
plt.close("all")

# decimate signal to make it more readible 
raw_ds = raw.copy().resample(100)

%matplotlib widget
scalings=1e-4

ax1 = plt.subplot(211)
plotTimeSeries(raw_ds.get_data().T, sfreq=raw_ds.info["sfreq"], ax=ax1, scalings=scalings, offset=offset)
plotAnnotations(annotations, color="red")
ax1.set_xlim(5145,5165)
ax2 = plt.subplot(212)
plotTimeSeries(raw_ds.get_data().T, sfreq=raw_ds.info["sfreq"], ax=ax2, scalings=scalings, offset=offset)
z = zoom_effect(ax1, ax2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# update manually the time axis
ax1.set_xlim(5145,5165) # in seconds

### (OPTIONAL) Enable widget

In [14]:
from ipywidgets import interact, FloatSlider
def update_axis(xmin, xmax):
    if xmin<xmax:
        ax1.set_xlim(xmin,xmax)

i=FloatSlider(min=raw.times[0], max=raw.times[-1], step=10, continuous_update=False)
ii=FloatSlider(min=raw.times[0], max=raw.times[-1], step=10, continuous_update=False)
from ipywidgets import FloatSlider
interact(update_axis,xmin=i, xmax=ii);

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='xmin', max=28924.995, step=…